QA2 is just QA1 (run1) but trained more, QA3 includes cleaned up answers and titles

# Prepare Dataset
by [samuel reams](twitter.com/spronkoid)


In [0]:
# just run this to download, or run the others to set it up
!wget https://gist.githubusercontent.com/spronkoid/e721c592b98384923db9a6df4d6cf5e5/raw/ce7ae132c26118d0fd2ebcf36a2915fdc8ad802d/QAdataset.txt

--2020-01-24 23:44:49--  https://gist.githubusercontent.com/spronkoid/e721c592b98384923db9a6df4d6cf5e5/raw/ce7ae132c26118d0fd2ebcf36a2915fdc8ad802d/QAdataset.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642328 (627K) [text/plain]
Saving to: ‘QAdataset.txt’

QAdataset.txt       100%[===================>] 627.27K  --.-KB/s    in 0.05s   

2020-01-24 23:44:49 (12.2 MB/s) - ‘QAdataset.txt’ saved [642328/642328]



original csv found here: https://www.kaggle.com/ananthu017/squad-csv-format<br>
thanks [ARES](https://www.kaggle.com/ananthu017/)!


In [0]:
import pandas
colnames = ['index', 'text', 'questions', 'answers']
data = pandas.read_csv('QA Dataset.csv', names=colnames)

In [0]:
dataset = ""

text=data.text.tolist()
questions=data.questions.tolist()
answers=data.answers.tolist()

c, q, a, e = "<|startoftext|>\n[CONTEXT]: ", "\n[QUESTION]:", "\n[ANSWER]:", "\n<|endoftext|>\n"
# print(eval(questions[1])[0])
index = 1
while(index<len(text)):
    dataset+=c+text[index]

    qindex = 1
    while(qindex<len(eval(questions[index]))):
        question = eval(questions[index])[qindex]
        questionS = ''.join(map(str, question))

        answer = eval(answers[index])[qindex]
        answerS = ''.join(map(str, answer))

        # ADDED FOR QA3
        answerI = answerS[:-21:]
        answerI = answerI[9 : : ]
        # REGULAR QA2

        dataset+=q+questionS
        dataset+=a+answerI
        # remove first 9, last 20
        # print("QINDEX: ", qindex)
        # print("INDEX: ", index)
        # print(questions[index])
        qindex+=1
    dataset+=e
    index+=1
print(dataset)

<|startoftext|>
[CONTEXT]: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
[QUESTION]:What areas did Beyonce compete in when she was growing up?
[ANSWER]:
[QUESTION]:When did Beyonce leave Destiny's Child and become a solo singer?
[ANSWER]:
[QUESTION]:In what city and state did Beyonce  grow up? 
[ANSWER]:
[QUESTION]:In which decade did Beyonce become

In [0]:
f = open("QAdataset.txt","w+")
f.write(dataset)
f.close()

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [0]:
!nvidia-smi

Fri Jan 24 23:45:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
gpt2.download_gpt2(model_name="345M")

Fetching checkpoint: 1.05Mit [00:00, 200Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 89.8Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 716Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:07, 201Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 517Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 94.6Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 190Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [0]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = "QAdataset.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
# we don't need this
# gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='345M',
              steps=4000,
              restore_from='fresh',
              run_name='QA',
              print_every=1,
              sample_every=2000,
              save_every=100
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/345M/model.ckpt
INFO:tensorflow:Restoring parameters from models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.24s/it]


dataset has 163789 tokens
Training...
[1 | 11.77] loss=1.95 avg=1.95
[2 | 13.27] loss=2.27 avg=2.11
[3 | 14.77] loss=1.89 avg=2.04
[4 | 16.27] loss=1.99 avg=2.02
[5 | 17.78] loss=1.86 avg=1.99
[6 | 19.28] loss=1.81 avg=1.96
[7 | 20.79] loss=1.99 avg=1.96
[8 | 22.30] loss=1.87 avg=1.95
[9 | 23.81] loss=1.56 avg=1.90
[10 | 25.32] loss=1.80 avg=1.89
[11 | 26.83] loss=1.86 avg=1.89
[12 | 28.34] loss=1.91 avg=1.89
[13 | 29.86] loss=1.95 avg=1.90
[14 | 31.39] loss=1.93 avg=1.90
[15 | 32.91] loss=1.95 avg=1.90
[16 | 34.45] loss=1.99 avg=1.91
[17 | 35.97] loss=1.68 avg=1.90
[18 | 37.51] loss=1.79 avg=1.89
[19 | 39.04] loss=1.85 avg=1.89
[20 | 40.58] loss=1.32 avg=1.86
[21 | 42.12] loss=1.76 avg=1.85
[22 | 43.67] loss=1.75 avg=1.85
[23 | 45.21] loss=1.76 avg=1.84
[24 | 46.76] loss=1.39 avg=1.82
[25 | 48.32] loss=1.97 avg=1.83
[26 | 49.87] loss=1.63 avg=1.82
[27 | 51.43] loss=1.58 avg=1.81
[28 | 52.99] loss=1.93 avg=1.81
[29 | 54.56] loss=1.88 avg=1.82
[30 | 56.12] loss=1.77 avg=1.81
[31 | 57.69

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='QA')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='QA')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='QA')

Loading checkpoint checkpoint/QA3-test1/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/QA3-test1/model-2000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess, run_name='QA')

[QUESTION]:What is the name given to the 20th century Great British revival Georgian architecture.
[ANSWER]:'Neo-Georgian',
[QUESTION]:What is the name of the house of the monarchs in power from 1714 and 1830?
[ANSWER]:'House of Hanover',
[QUESTION]:What was the name of the 20th century revival of Georgian architecture in the United States?
[ANSWER]:'Romance of the Fourteenth',
[QUESTION]:What was the name of the 19th century revival in Great Britian?
[ANSWER]:'Romance of the Seventeenth',
[QUESTION]:What type of architecture was current before 1714?
[ANSWER]:
[QUESTION]:What was the name of the 19th century revival in Mexico?
[ANSWER]:
[QUESTION]:What was the name of the 20th century revival in the United States?
[ANSWER]:
<|endoftext|>
<|startoftext|>
[CONTEXT]: The Republic of Liberia, beginning as a settlement of the American Colonization Society (ACS), declared its independence on July 26, 1847. The United States did not recognize Liberia's independence until during the American C

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
ctx = input("Context: ")
qst = input("Question: ")
ans = "[ANSWER]:"
pre = '<|startoftext|>\n[CONTEXT]: ' + ctx + "\n[QUESTION]:" + qst + "\n" + ans

Context: The Battle of Mikatagahara (三方ヶ原の戦い, Mikatagahara no tatakai) (January 25, 1573;[1] Tōtōmi Province, Japan) was one of the most famous battles of Takeda Shingen's campaigns, and one of the best demonstrations of his cavalry-based tactics. It was also one of Tokugawa Ieyasu's worst defeats, and complete disaster was only narrowly averted. 
Question: Who lost?


In [0]:
gpt2.generate(sess,
              length=5,
              temperature=0.7,
              prefix=pre,
              nsamples=10,
              batch_size=10,
              run_name="QA",
              )

<|startoftext|>
[CONTEXT]: The Battle of Mikatagahara (三方ヶ原の戦い, Mikatagahara no tatakai) (January 25, 1573;[1] Tōtōmi Province, Japan) was one of the most famous battles of Takeda Shingen's campaigns, and one of the best demonstrations of his cavalry-based tactics. It was also one of Tokugawa Ieyasu's worst defeats, and complete disaster was only narrowly averted. 
[QUESTION]:Who lost?
[ANSWER]:'Kiyohito
<|startoftext|>
[CONTEXT]: The Battle of Mikatagahara (三方ヶ原の戦い, Mikatagahara no tatakai) (January 25, 1573;[1] Tōtōmi Province, Japan) was one of the most famous battles of Takeda Shingen's campaigns, and one of the best demonstrations of his cavalry-based tactics. It was also one of Tokugawa Ieyasu's worst defeats, and complete disaster was only narrowly averted. 
[QUESTION]:Who lost?
[ANSWER]:'Tokugawa I
<|startoftext|>
[CONTEXT]: The Battle of Mikatagahara (三方ヶ原の戦い, Mikatagahara no tatakai) (January 25, 1573;[1] Tōtōmi Province, Japan) was one of the most famous battles of Takeda Sh

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
# custom code again
ctx = input("Context: ")
qst = input("Question: ")
ans = "[ANSWER]:"
pre = '<|startoftext|>\n[CONTEXT]: ' + ctx + "\n[QUESTION]:" + qst + "\n" + ans

In [0]:
# gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())
gen_file = 'predictions.txt'

gpt2.generate_to_file(sess,
    destination_path=gen_file,
    length=5,
    temperature=1,
    prefix=pre,
    nsamples=10,
    batch_size=10,
    run_name="QA",
    truncate='<|endoftext|>',
    #   include_prefix=False
    )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

## LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Use
[Samuel Reams](github.com/spronkoid)

In [0]:
ctx = input("Context: ")
qst = input("Question: ")
ans = "[ANSWER]:"
pre = '<|startoftext|>\n[CONTEXT]: ' + ctx + "\n[QUESTION]:" + qst + "\n" + ans

Context: Alice in Wonderland is a 1951 American animated musical fantasy-adventure film produced by Walt Disney Productions and based on the Alice books by Lewis Carroll. The 13th release of Disney's animated features, the film premiered in London on July 26, 1951, and in New York City on July 28, 1951. The film features the voices of Kathryn Beaumont as Alice, Sterling Holloway as the Cheshire Cat, Verna Felton as the Queen of Hearts, and Ed Wynn as the Mad Hatter.
Question: Who produced it?


In [0]:
# generate five samples to see five different ideas that it has, just grab the one it says most, idk it's really jank
gpt2.generate(sess,
    length=5,
    temperature=1,
    prefix=pre,
    nsamples=10,
    batch_size=10,
    run_name="QA",
)

<|startoftext|>
[CONTEXT]: Alice in Wonderland is a 1951 American animated musical fantasy-adventure film produced by Walt Disney Productions and based on the Alice books by Lewis Carroll. The 13th release of Disney's animated features, the film premiered in London on July 26, 1951, and in New York City on July 28, 1951. The film features the voices of Kathryn Beaumont as Alice, Sterling Holloway as the Cheshire Cat, Verna Felton as the Queen of Hearts, and Ed Wynn as the Mad Hatter.
[QUESTION]:Who produced it?
[ANSWER]:'Walt Disney Productions
<|startoftext|>
[CONTEXT]: Alice in Wonderland is a 1951 American animated musical fantasy-adventure film produced by Walt Disney Productions and based on the Alice books by Lewis Carroll. The 13th release of Disney's animated features, the film premiered in London on July 26, 1951, and in New York City on July 28, 1951. The film features the voices of Kathryn Beaumont as Alice, Sterling Holloway as the Cheshire Cat, Verna Felton as the Queen of

### generate to file

In [0]:
# gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())
gen_file = 'predictions.txt'

gpt2.generate_to_file(sess,
    destination_path=gen_file,
    length=5,
    temperature=1,
    prefix=pre,
    nsamples=10,
    batch_size=10,
    run_name="QA3-test1",
    )